In [1]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
import datetime

In [137]:
class TwStock:
#     對照表
    m_mapping = {'上市':'sii','上櫃':'otc'}
    base_url = 'https://mops.twse.com.tw/mops/web/'
    p_mapping = {'損益表':base_url+'t163sb04'
                ,'資產負債表':base_url+'t163sb05'
                ,'營益分析':base_url+'t163sb06'
                ,'財務結構分析':base_url+'t51sb02'}
    old_p_mapping = {'損益表':base_url+'t51sb08'
                ,'資產負債表':base_url+'t51sb07'
                ,'營益分析':base_url+'t51sb06'
                ,'財務結構分析':base_url+'ajax_t51sb02'}

    def __init__(self, year, season, mkt_type, purpose, \
                 filter_ind=True, m_mapping=m_mapping, p_mapping=p_mapping, old_p_mapping=old_p_mapping):
        year = year if year < 1000 else year-1911
        self.year = year
        self.season = '0'+str(season) if type(season)==int else season
        self.mkt_type = m_mapping[mkt_type] if mkt_type in m_mapping else None
        self.purpose = purpose
        p_mapping = p_mapping if self.year>=102 else old_p_mapping
        self.url = p_mapping[self.purpose] if self.purpose in p_mapping else None
        self.filter_ind=filter_ind
    
    def add_raw(self):
        form = {'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':self.mkt_type,
            'year':str(self.year),
            'season':self.season
            }
        if self.year>=102:
            form['ifrs']='Y'
        else:
            form['ifrs']='N'
            
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}    
            
        r = requests.post(self.url, form, headers=headers)
        r.encoding = 'utf8'
        dfs = pd.read_html(StringIO(r.text))
        self.raw_data = dfs
        
#     營益分析
    def unify_incm_ratio(self):
        dfs = [i for i in self.raw_data if i.shape[0]>5]
        dfs = [i for i in dfs if i.shape[1]>5]
        data = dfs[0]
        data.columns = ['公司代號','公司名稱','營業收入(百萬元)','毛利率(%)','營業利益率(%)','稅前純益率(%)','稅後純益率(%)']
        data = data[data['公司代號']!='公司代號']
#         data.rename(columns={'毛利率(%)(營業毛利)/(營業收入)':'毛利率(%)'
#                      ,'營業利益率(%)(營業利益)/(營業收入)':'營業利益率(%)'
#                      ,'稅前純益率(%)(稅前純益)/(營業收入)':'稅前純益率(%)'
#                      ,'稅後純益率(%)(稅後純益)/(營業收入)':'稅後純益率(%)'}, inplace=True)
        data = data.reset_index().drop('index', axis=1)
        self.data = data

#     財務結構分析
    def unify_baln_ratio(self, smp=True):
        dfs = [i for i in self.raw_data if i.shape[0]>5]
        dfs = [i for i in dfs if i.shape[1]>5]
        data = dfs[0]
        data.columns = data.columns.get_level_values(1)
        data = data[data['公司代號']!='公司代號']
        data = data.reset_index().drop('index', axis=1)
        if smp==True :
            clms = list(data.columns)
            clms = [i for i in clms if i not in \
                   ['長期資金佔不動產、廠房及設備比率(%)','不動產、廠房及設備週轉率(次)','長期資金佔固定資產比率(%)'
                    ,'平均收現日數','平均售貨日數','平均銷貨日數','純益率(%)','應收款項收現日數'
                    ,'固定資產週轉率(次)','稅前純益佔實收資本比率(%)','營業利益佔實收資本比率(%)']]
            data = data[clms]
            if '股東權益報酬率(%)' in data.columns:
                data.rename(columns={'股東權益報酬率(%)':'權益報酬率(%)'}, inplace=True)
            data.rename(columns={'負債佔資產比率(%)':'負債比率(%)'}, inplace=True)
        self.data = data

# ifrs 後

$ 上市 / 營益分析 $

In [152]:
x = TwStock(2015, 1, '上市', '營益分析')
x.add_raw()
x.unify_incm_ratio()
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,21440.14,13.34,7.90,6.86,4.55
1,1102,亞泥,15362.53,10.62,6.75,12.01,10.04
2,1103,嘉泥,671.36,-11.27,-27.67,-31.21,-27.30
3,1104,環泥,1327.43,8.30,1.85,20.32,19.73
4,1108,幸福,1290.90,20.88,15.37,14.48,12.01


In [153]:
x = TwStock(2015, 2, '上市', '營益分析')
x.add_raw()
x.unify_incm_ratio()
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,46434.33,14.60,9.93,10.15,7.97
1,1102,亞泥,32252.76,10.63,6.20,15.89,14.19
2,1103,嘉泥,1408.79,-5.92,-18.04,-0.68,-4.80
3,1104,環泥,2558.28,8.43,1.90,26.25,23.91
4,1108,幸福,2636.96,21.77,15.80,14.95,11.71


In [154]:
x = TwStock(2015, 3, '上市', '營益分析')
x.add_raw()
x.unify_incm_ratio()
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,69118.30,14.59,9.56,8.51,6.79
1,1102,亞泥,49113.91,10.04,5.81,10.36,8.97
2,1103,嘉泥,1957.61,-6.70,-20.47,-18.89,-19.65
3,1104,環泥,3770.78,9.52,2.56,26.07,24.13
4,1108,幸福,3765.70,20.02,14.85,14.38,10.61


In [155]:
x = TwStock(2015, 4, '上市', '營益分析')
x.add_raw()
x.unify_incm_ratio()
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,93679.08,15.51,10.33,9.25,7.39
1,1102,亞泥,66287.48,10.65,6.09,10.28,7.44
2,1103,嘉泥,2826.52,-3.81,14.47,6.90,7.55
3,1104,環泥,5163.75,10.41,3.36,28.50,26.78
4,1108,幸福,5021.35,18.05,12.43,12.27,9.22


In [63]:
x = TwStock(2015, 1, '上櫃', '營益分析')
x.add_raw()
x.unify_incm_ratio()

In [64]:
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1258,其祥-KY,324.18,32.33,10.06,11.79,9.26
1,1259,安心,1053.39,25.79,1.75,2.16,1.51
2,1264,德麥,937.93,36.67,15.31,15.33,12.41
3,1333,恩得利,226.08,17.71,-3.56,2.64,1.81
4,1336,台翰,418.88,2.63,-10.71,-9.81,-8.17


In [6]:
x = TwStock(2015, None, '上市', '財務結構分析')

In [7]:
x.__dict__

{'year': 104,
 'season': None,
 'mkt_type': 'sii',
 'purpose': '財務結構分析',
 'url': 'https://mops.twse.com.tw/mops/web/t51sb02',
 'filter_ind': True}

In [8]:
x.add_raw()

In [10]:
x.unify_baln_ratio()

In [12]:
x.data.head()

,公司代號,公司簡稱,負債比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),存貨週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,46.95,133.06,119.28,6.07,4.01,8.15,0.32,2.85,4.36,1.56,31.97,109.73,3.19
1,1102,亞泥,42.66,81.02,69.80,5.14,3.55,7.08,0.24,2.28,3.12,1.55,20.95,112.25,2.97
2,1103,嘉泥,48.70,699.93,609.52,186.35,6.34,1.75,0.10,1.31,1.28,0.34,19.88,59.60,0.24
3,1104,環泥,19.88,123.51,103.82,101.84,4.26,13.93,0.26,7.09,8.70,2.22,34.43,59.18,1.09
4,1108,幸福,38.32,285.67,81.91,15.46,4.91,1.16,0.64,6.14,9.78,1.16,55.80,221.19,4.35


In [67]:
x = TwStock(2015, None, '上櫃', '財務結構分析')
x.add_raw()
x.unify_baln_ratio()

In [69]:
x.data.head()

,公司代號,公司簡稱,負債比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),存貨週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1258,其祥-KY,50.33,205.87,192.60,24.73,10.87,47.45,1.48,10.69,17.60,3.79,52.33,NaN,14.04
1,1259,安心,36.50,229.18,222.53,63061.00,80.98,73.13,1.75,3.41,5.27,4.05,42.80,127.72,0.09
2,1264,德麥,22.55,346.51,263.74,5465.08,4.75,5.29,1.47,16.71,22.01,13.33,70.76,103.33,5.29
3,1268,漢來美食,56.34,114.89,46.52,37.11,34.01,31.50,1.19,8.45,21.94,6.13,32.11,195.33,17.97
4,1333,恩得利,56.60,91.01,66.98,-0.02,2.30,5.27,0.72,-1.24,-6.76,-0.33,1.86,42.53,1.92


# ifrs 前

$ 上市 / 營益分析 $

In [85]:
x = TwStock(2008, 1, '上市', '營益分析')

In [86]:
x.add_raw()

In [87]:
x.unify_incm_ratio()

In [88]:
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,5959,6.72,1.82,21.77,21.52
1,1102,亞泥,2638,18.59,11.68,111.53,108.53
2,1103,嘉泥,618,13.00,4.84,36.16,36.09
3,1104,環泥,699,7.02,-5.94,26.48,26.56
4,1108,幸福,807,12.64,4.40,14.41,12.27


$ 上櫃 / 營益分析 $

In [93]:
x = TwStock(2008, 1, '上櫃', '營益分析')
x.add_raw()
x.unify_incm_ratio()

In [94]:
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1333,恩得利,187,12.02,-10.99,-42.78,-35.27
1,1336,台翰,315,21.92,16.42,-8.02,-7.86
2,1565,精華,522,57.69,36.57,32.66,24.73
3,1569,濱川,142,10.47,1.71,8.88,3.57
4,1570,力肯,137,24.12,0.40,-1.15,1.42


$ 上市 / 財務結構分析 $

In [139]:
x = TwStock(2008, None, '上市', '財務結構分析')

In [140]:
x.__dict__

{'year': 97,
 'season': None,
 'mkt_type': 'sii',
 'purpose': '財務結構分析',
 'url': 'https://mops.twse.com.tw/mops/web/ajax_t51sb02',
 'filter_ind': True}

In [141]:
x.add_raw()

In [142]:
x.unify_baln_ratio()

In [143]:
x.data.head()

,公司代號,公司簡稱,負債比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),存貨週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,27.59,87.31,70.65,11.49,5.75,10.02,0.24,5.87,7.44,1.75,53.80,117.75,1.54
1,1102,亞泥,32.75,174.06,130.33,13.84,8.88,4.41,0.11,8.01,10.67,2.52,134.19,139.43,0.28
2,1103,嘉泥,34.31,134.63,125.52,285.88,6.41,18.62,0.11,1.37,1.41,0.39,14.12,37.99,1.64
3,1104,環泥,31.07,83.66,43.13,1.64,4.55,4.83,0.17,0.75,0.50,0.09,5.92,51.15,0.57
4,1108,幸福,50.19,82.47,52.23,0.32,5.92,4.18,0.39,0.22,-0.76,-0.08,18.98,89.22,3.87


$ 上櫃 / 財務結構分析 $

In [144]:
x = TwStock(2008, None, '上櫃', '財務結構分析')
x.add_raw()
x.unify_baln_ratio()

In [147]:
x.data.head()

,公司代號,公司簡稱,負債比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),存貨週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1333,恩得利,58.75,173.85,147.33,-54.90,2.47,4.06,0.38,-37.11,-67.19,-7.96,18.09,45.97,4.17
1,1336,台翰,19.21,287.75,281.74,40.52,5.90,30.61,0.58,2.64,3.18,0.91,54.48,342.11,4.27
2,1565,精華,30.59,301.80,238.84,71.81,7.08,2.69,0.77,21.43,30.58,11.16,145.23,106.49,29.65
3,1569,濱川,40.82,100.57,69.75,6.74,2.97,80.61,0.35,4.10,5.75,1.08,-3.90,-7.10,-3.03
4,1570,力肯,23.07,223.86,139.12,-61.82,3.04,2.90,0.62,-7.98,-10.48,-1.31,59.62,98.92,11.91
